In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.externals import joblib
#Para guardar    joblib.dump(grid, 'grid.pkl')
#Para cargar     clf = joblib.load('grid.pkl')

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

# Pruebo guardar un gridSearch

In [7]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,10]
samples_split = [2]
features = [1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, \
              "min_samples_split" : samples_split, "max_features" : features}

iteraciones_cross_validation = 3
arbol = DecisionTreeRegressor()

In [8]:
grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = -1, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_

print("Precision: {:.2f} \n Parametros = {}".format(score,mejores_parametros))

Precision: 75.01 
 Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 1.0, 'min_samples_split': 2}


In [9]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

In [10]:
resultado.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.494666e+05
std,4.290968e+04,6.646246e+05
min,3.632000e+03,5.000000e+03
25%,2.271809e+06,9.700000e+04
50%,2.290284e+06,1.500000e+05
75%,2.309395e+06,2.600000e+05
max,2.332338e+06,4.654544e+07


In [11]:
resultado.head()

,id,price_usd
0,3632,1099900.0
1,3633,159000.0
2,2263404,321800.0
3,2263405,78000.0
4,2263406,196000.0


In [12]:
grid

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'criterion': ['mse', 'friedman_mse'], 'max_depth': [None, 10], 'min_samples_split': [2], 'max_features': [1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [13]:
joblib.dump(grid, 'grid.pkl')

['grid.pkl']

## Ahora lo cargo, haciendo restart del kernel

In [5]:
clf = joblib.load('grid.pkl') 

In [6]:
clf

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'criterion': ['mse', 'friedman_mse'], 'max_depth': [None, 10], 'min_samples_split': [2], 'max_features': [1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [7]:
score = clf.best_score_ * 100
mejores_parametros = clf.best_params_

print("Precision: {:.2f} \n Parametros = {}".format(score,mejores_parametros))

Precision: 75.01 
 Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 1.0, 'min_samples_split': 2}


In [10]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: clf.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

In [11]:
resultado.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.494666e+05
std,4.290968e+04,6.646246e+05
min,3.632000e+03,5.000000e+03
25%,2.271809e+06,9.700000e+04
50%,2.290284e+06,1.500000e+05
75%,2.309395e+06,2.600000e+05
max,2.332338e+06,4.654544e+07


In [12]:
resultado.head()

,id,price_usd
0,3632,1099900.0
1,3633,159000.0
2,2263404,321800.0
3,2263405,78000.0
4,2263406,196000.0


## Da los mismos resultados!!!